In [ ]:
#pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.7 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
from transformers import TFViTForImageClassification, ViTFeatureExtractor
from datasets import load_dataset
import os
import datasets

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
os.mkdir("/content/drive/MyDrive/EE992/ViT/")
path = "/content/drive/MyDrive/EE992/ViT/"


In [3]:
model_name = "google/vit-base-patch16-224-in21k"
feature_extractor = ViTFeatureExtractor.from_pretrained(model_name)
model = TFViTForImageClassification.from_pretrained(model_name)

/Users/frankconway/miniconda3/envs/tensorflow/lib/python3.10/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(
2025-03-18 14:16:14.621173: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Max
2025-03-18 14:16:14.621289: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 32.00 GB
2025-03-18 14:16:14.621305: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 10.67 GB
2025-03-18 14:16:14.621362: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-03-18 14:16:14.621402: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task

In [ ]:

# dataset = datasets.load_dataset("cifar100")

# img_class_labels = dataset['train'].features['fine_label'].names
# # img_class_labels
# # we are also renaming our label col to labels to use `.to_tf_dataset` later
# dataset = dataset.rename_column("fine_label", "labels")

# # Create a preprocessing function
# def preprocess_function(examples):
#     images = examples["img"]
#     inputs = feature_extractor(images=images, return_tensors="tf")
#     inputs["labels"] = examples["labels"]
#     return inputs


# # Apply preprocessing
# processed_dataset = dataset.map(preprocess_function, batched=True)


In [ ]:
# # test size will be 15% of train dataset
# test_size=.15

# # Apply train_test_split to the 'train' split of the dataset
# processed_dataset = processed_dataset["train"].train_test_split(test_size=test_size)
# # Update the 'train' and 'test' splits in the DatasetDict
# processed_dataset["train"] = processed_dataset["train"]
# processed_dataset["test"] = processed_dataset["test"]
# processed_dataset


# test_size = 0.15
# train_val_split = processed_dataset["train"].train_test_split(test_size=test_size)

# # Create the final dataset dictionary
# final_dataset = datasets.DatasetDict({
#     "train": train_val_split["train"],
#     "validation": train_val_split["test"],
#     "test": train_val_split["test"]
# })

# # Convert to TensorFlow datasets
# tf_train_dataset = processed_dataset["train"].to_tf_dataset(
#     columns=["pixel_values", "labels"],
#     shuffle=True,
#     batch_size=16,
#     collate_fn=None,
# )

# tf_validation_dataset = processed_dataset["test"].to_tf_dataset(
#     columns=["pixel_values", "labels"],
#     shuffle=False,
#     batch_size=16,
#     collate_fn=None,
# )

# tf_test_dataset = processed_dataset["test"].to_tf_dataset(
#     columns=["pixel_values", "labels"],
#     shuffle=False,
#     batch_size=16,
#     collate_fn=None,
# )

In [ ]:
# tf_train_dataset = processed_dataset["train"].to_tf_dataset(
#     columns=["pixel_values", "labels"],
#     shuffle=True,
#     batch_size=16,
#     collate_fn=None,
# )

# tf_validation_dataset = processed_dataset["test"].to_tf_dataset(
#     columns=["pixel_values", "labels"],
#     shuffle=False,
#     batch_size=16,
#     collate_fn=None,
# )

In [10]:
from transformers import ViTImageProcessor
import datasets

# Load CIFAR-100 dataset
dataset = datasets.load_dataset("cifar100")
img_class_labels = dataset['train'].features['fine_label'].names
# Rename column to labels for .to_tf_dataset later
dataset = dataset.rename_column("fine_label", "labels")

# Initialize the ViT image processor for 224x224 resizing and normalization
feature_extractor = ViTImageProcessor(
    size=224,
    do_resize=True,
    do_normalize=True,
    image_mean=[0.5, 0.5, 0.5],
    image_std=[0.5, 0.5, 0.5],
)

# Create a preprocessing function
def preprocess_function(examples):
    images = examples["img"]
    # Process images using ViTImageProcessor
    inputs = feature_extractor(images=images, return_tensors="tf")
    inputs["labels"] = examples["labels"]
    return inputs

# Apply preprocessing
processed_dataset = dataset.map(preprocess_function, batched=True)

# Split into train/validation sets
test_size = 0.15
train_val_split = processed_dataset["train"].train_test_split(test_size=test_size)

# Create the final dataset dictionary
final_dataset = datasets.DatasetDict({
    "train": train_val_split["train"],
    "validation": train_val_split["test"],
    "test": processed_dataset["test"]  # Use the original test set
})





# # Number of classes in CIFAR-100
# num_classes = 100

# # Function to convert sparse labels to one-hot encoded (categorical)
# def sparse_to_categorical(features):
#     features["categorical_labels"] = tf.one_hot(features["labels"], depth=num_classes)
#     return features

# # Convert to TensorFlow datasets with categorical labels
# tf_train_dataset = final_dataset["train"].to_tf_dataset(
#     columns=["pixel_values", "labels"],
#     shuffle=True,
#     batch_size=16,
#     collate_fn=None,
# ).map(sparse_to_categorical)

# tf_validation_dataset = final_dataset["validation"].to_tf_dataset(
#     columns=["pixel_values", "labels"],
#     shuffle=False,
#     batch_size=16,
#     collate_fn=None,
# ).map(sparse_to_categorical)

# tf_test_dataset = final_dataset["test"].to_tf_dataset(
#     columns=["pixel_values", "labels"],
#     shuffle=False,
#     batch_size=16,
#     collate_fn=None,
# ).map(sparse_to_categorical)

# Convert to TensorFlow datasets
tf_train_dataset = final_dataset["train"].to_tf_dataset(
    columns=["pixel_values", "labels"],
    shuffle=True,
    batch_size=16,
    collate_fn=None,
)

tf_validation_dataset = final_dataset["validation"].to_tf_dataset(
    columns=["pixel_values", "labels"],
    shuffle=False,
    batch_size=16,
    collate_fn=None,
)

tf_test_dataset = final_dataset["test"].to_tf_dataset(
    columns=["pixel_values", "labels"],
    shuffle=False,
    batch_size=16,
    collate_fn=None,
)

In [ ]:
final_dataset["test"].features

{'img': Image(mode=None, decode=True, id=None),
 'labels': ClassLabel(names=['apple', 'aquarium_fish', 'baby', 'bear', 'beaver', 'bed', 'bee', 'beetle', 'bicycle', 'bottle', 'bowl', 'boy', 'bridge', 'bus', 'butterfly', 'camel', 'can', 'castle', 'caterpillar', 'cattle', 'chair', 'chimpanzee', 'clock', 'cloud', 'cockroach', 'couch', 'cra', 'crocodile', 'cup', 'dinosaur', 'dolphin', 'elephant', 'flatfish', 'forest', 'fox', 'girl', 'hamster', 'house', 'kangaroo', 'keyboard', 'lamp', 'lawn_mower', 'leopard', 'lion', 'lizard', 'lobster', 'man', 'maple_tree', 'motorcycle', 'mountain', 'mouse', 'mushroom', 'oak_tree', 'orange', 'orchid', 'otter', 'palm_tree', 'pear', 'pickup_truck', 'pine_tree', 'plain', 'plate', 'poppy', 'porcupine', 'possum', 'rabbit', 'raccoon', 'ray', 'road', 'rocket', 'rose', 'sea', 'seal', 'shark', 'shrew', 'skunk', 'skyscraper', 'snail', 'snake', 'spider', 'squirrel', 'streetcar', 'sunflower', 'sweet_pepper', 'table', 'tank', 'telephone', 'television', 'tiger', 'tractor

In [ ]:

model_name = "google/vit-base-patch16-224-in21k"

id2label = {str(i): label for i, label in enumerate(img_class_labels)}
label2id = {v: k for k, v in id2label.items()}

# load pre-trained ViT model
model = TFViTForImageClassification.from_pretrained(
    model_name,
    num_labels=len(img_class_labels),
    id2label=id2label,
    label2id=label2id,
)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFViTForImageClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing TFViTForImageClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFViTForImageClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFViTForImageClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import pandas as pd

lr = [3e-5, 1e-5]


for LR in lr:

    model_name = "ViT_"+str(LR)

    model_name = "google/vit-base-patch16-224-in21k"

    id2label = {str(i): label for i, label in enumerate(img_class_labels)}
    label2id = {v: k for k, v in id2label.items()}


    # load pre-trained ViT model
    model = TFViTForImageClassification.from_pretrained(
        model_name,
        num_labels=len(img_class_labels),
        id2label=id2label,
        label2id=label2id,
    )

    # Define optimizer and loss
    optimizer = tf.keras.optimizers.AdamW(learning_rate=LR)
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    checkpoint = tf.keras.callbacks.ModelCheckpoint(path+model_name+".keras", save_best_only=True, monitor='val_loss')

    # Compile the model
    model.compile(optimizer=optimizer, loss=loss, metrics=["accuracy"])

    # Train the model
    history = model.fit(
        tf_train_dataset,
        validation_data=tf_validation_dataset,
        epochs=10,
        callbacks=[early_stopping, checkpoint]
    )

    # convert history to DataFrame
    history_df = pd.DataFrame(history.history)

    history_df.to_csv(path+model_name+".csv", index=False)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFViTForImageClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing TFViTForImageClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFViTForImageClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFViTForImageClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
2657/2657 [==============================] - 383s 129ms/step - loss: 2.0388 - accuracy: 0.8027 - val_loss: 0.6566 - val_accuracy: 0.8835
Epoch 2/10
2657/2657 [==============================] - 345s 130ms/step - loss: 0.3216 - accuracy: 0.9364 - val_loss: 0.3817 - val_accuracy: 0.9080
Epoch 3/10
2657/2657 [==============================] - 343s 129ms/step - loss: 0.1487 - accuracy: 0.9647 - val_loss: 0.3657 - val_accuracy: 0.9053
Epoch 4/10
2657/2657 [==============================] - 338s 127ms/step - loss: 0.0918 - accuracy: 0.9773 - val_loss: 0.4319 - val_accuracy: 0.8927
Epoch 5/10
2657/2657 [==============================] - 337s 127ms/step - loss: 0.0680 - accuracy: 0.9823 - val_loss: 0.4188 - val_accuracy: 0.9003
Epoch 6/10
2657/2657 [==============================] - 335s 126ms/step - loss: 0.0522 - accuracy: 0.9858 - val_loss: 0.4071 - val_accuracy: 0.9013


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFViTForImageClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing TFViTForImageClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFViTForImageClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFViTForImageClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
2657/2657 [==============================] - 380s 128ms/step - loss: 3.4746 - accuracy: 0.7039 - val_loss: 2.3883 - val_accuracy: 0.8705
Epoch 2/10
2657/2657 [==============================] - 348s 131ms/step - loss: 1.5662 - accuracy: 0.9068 - val_loss: 1.0317 - val_accuracy: 0.9003
Epoch 3/10
2657/2657 [==============================] - 346s 130ms/step - loss: 0.5819 - accuracy: 0.9478 - val_loss: 0.5395 - val_accuracy: 0.9079
Epoch 4/10
2657/2657 [==============================] - 345s 130ms/step - loss: 0.2412 - accuracy: 0.9687 - val_loss: 0.4162 - val_accuracy: 0.9068
Epoch 5/10
2657/2657 [==============================] - 346s 130ms/step - loss: 0.1258 - accuracy: 0.9790 - val_loss: 0.3894 - val_accuracy: 0.9083
Epoch 6/10
2657/2657 [==============================] - 348s 131ms/step - loss: 0.0755 - accuracy: 0.9859 - val_loss: 0.3787 - val_accuracy: 0.9093
Epoch 7/10
2657/2657 [==============================] - 341s 128ms/step - loss: 0.0515 - accuracy: 0.9897 - val_

In [ ]:
# Import necessary metrics modules
from tensorflow.keras.metrics import Precision, Recall, F1Score

# Define optimizer and loss
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=[
        "accuracy"]
)

# Train the model
history = model.fit(
    tf_train_dataset,
    validation_data=tf_validation_dataset,
    epochs=3
)


ValueError: Could not interpret optimizer identifier: <keras.src.optimizers.adam.Adam object at 0x16a340d00>

In [ ]:
# Define optimizer and loss
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

from tensorflow.keras import backend as K
# Compile the model
# model.compile(optimizer=optimizer, loss=loss, metrics=["accuracy",tf.keras.metrics.Precision(name='precision') ])

def f1(y_true, y_pred):
    def recall_m(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = TP / (Positives + K.epsilon())
        return recall

    def precision_m(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Pred_Positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = TP / (Pred_Positives + K.epsilon())
        return precision

    precision, recall = precision_m(y_true, y_pred), recall_m(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy", f1]
)



# Train the model
history = model.fit(
    tf_train_dataset,
    validation_data=tf_validation_dataset,
    epochs=3
)

ValueError: Could not interpret optimizer identifier: <keras.src.optimizers.adam.Adam object at 0x174793280>

In [ ]:
# Import necessary metrics modules
from tensorflow.keras.metrics import Precision, Recall
import tensorflow as tf

# Define optimizer and loss
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Compile the model
model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=[
        "accuracy",
        # For precision and recall with sparse labels, convert predictions to classes first
        tf.keras.metrics.Precision(name='precision', top_k=1, class_id=None),
        tf.keras.metrics.Recall(name='recall', top_k=1, class_id=None)
    ]
)

# Train the model
history = model.fit(
    tf_train_dataset,
    validation_data=tf_validation_dataset,
    epochs=3
)

AttributeError: 'Variable' object has no attribute '_distribute_strategy'

In [9]:
dataset = datasets.load_dataset("cifar100")

img_class_labels = dataset['train'].features['fine_label'].names
# img_class_labels
# we are also renaming our label col to labels to use `.to_tf_dataset` later
dataset = dataset.rename_column("fine_label", "labels")

# Create a preprocessing function
def preprocess_function(examples):
    images = examples["img"]
    inputs = feature_extractor(images=images, return_tensors="tf")
    inputs["labels"] = examples["labels"]
    return inputs


# Initialize the ViT image processor for 224x224 resizing and normalization
feature_extractor = ViTImageProcessor(
    size=224,
    do_resize=True,
    do_normalize=True,
    image_mean=[0.5, 0.5, 0.5],
    image_std=[0.5, 0.5, 0.5],
)

# Create a preprocessing function
def preprocess_function(examples):
    images = examples["img"]
    # Process images using ViTImageProcessor
    inputs = feature_extractor(images=images, return_tensors="tf")
    inputs["labels"] = examples["labels"]
    return inputs

# Apply preprocessing
processed_dataset = dataset.map(preprocess_function, batched=True)

NameError: name 'ViTImageProcessor' is not defined

In [ ]:
processed_dataset

In [11]:
id2label = {str(i): label for i, label in enumerate(img_class_labels)}
label2id = {v: k for k, v in id2label.items()}

# load pre-trained ViT model
model = TFViTForImageClassification.from_pretrained(
    "ViT_3e-05.keras", return_dict=False,
    num_labels=len(img_class_labels),
    id2label=id2label,
    label2id=label2id,
)

OSError: It looks like the config file at 'ViT_3e-05.keras' is not a valid JSON file.

In [16]:
from tensorflow.keras.models import load_model

# Load your existing Keras model
model = load_model("ViT_3e-05.keras")

ValueError: Expected object to be an instance of `KerasSaveable`, but got <transformers.models.vit.modeling_tf_vit.TFViTForImageClassification object at 0x36dd241c0> of type <class 'transformers.models.vit.modeling_tf_vit.TFViTForImageClassification'>

In [ ]:
tf_test_dataset = processed_dataset["test"].to_tf_dataset(
    columns=["pixel_values", "labels"],
    shuffle=True,
    batch_size=16,
    collate_fn=None,
)

In [15]:
import tensorflow as tf

# Load the model
from tensorflow.keras.layers import LayerNormalization

model = tf.keras.models.load_model('ViT_3e-05.keras', custom_objects={'LayerNormalization': feature_extractor})



# Print model summary
model.summary()


ValueError: Expected object to be an instance of `KerasSaveable`, but got <transformers.models.vit.modeling_tf_vit.TFViTForImageClassification object at 0x36d85d720> of type <class 'transformers.models.vit.modeling_tf_vit.TFViTForImageClassification'>

In [ ]:
# model_name = "google/vit-base-patch16-224-in21k"

# tokenizer = ViTFeatureExtractor.from_pretrained(model_name)
# model = ViTForImageClassification.from_pretrained("ViT_3e-05.keras", return_dict=False)


model.predict(tf_test_dataset)

In [ ]:
model.evaluate(tf_test_dataset)

In [ ]:
pwd

'/Users/frankconway/Library/CloudStorage/OneDrive-Personal/Strathclyde/Strathclyde/Year5/EE992/Project/Vit'